In [1]:

# coding: utf-8

# In[392]:

from __future__ import division
import numpy as np
import time

#from reportlab.lib.pagesizes import A4
#from reportlab.pdfgen import canvas
#from reportlab.platypus import Table,TableStyle


class AHP:
    def __init__(self):
        self.variable = {}
    #归一化
    def norm(self, m):
        if m.shape[0] == 1:
            return np.array(m)
        d1 = np.array(m)
#         d2 = (d1 - np.min(d1, axis=0)+1e-7)/(np.max(d1, axis=0) - np.min(d1, axis=0)+1e-7)
        d2 = d1/np.sum(d1,axis=0)
        return d2
    
    def getScoreTable(self,m):
        s = m.shape[0]
        scoreTable = np.zeros((s,s))
        for i in range(s):
            for j in range(s):
                scoreTable[i][j] = m[i]/m[j];
        return scoreTable

    def calculate_weight(self, columns_names, judgment_matrix):
        '''
        和法求权重
        columns_names:a list of data names used
        judgment_matrix:the judgment_matrix of AHP given by expert
        '''
        ########basic parameters of input######
        nrow, ncol = judgment_matrix.shape
        if nrow != ncol:
            print ("the matrix is NOT square")
            return
        if len(columns_names) != nrow:
            print ("columns_names NOT MATCH the shape of judgment_matrix")
            return
        ####string version of judgment matrix####
        str_matrix = []
        for i in range(nrow):
            str_i = []
            for j in range(ncol):
                p = judgment_matrix[i, j]
                if judgment_matrix[i, j] >= 1:
                    str_i.append(str(int(p)).center(5))
                else:
                    str_i.append(('1/' + str(int(1.0 / p))).center(5))
            str_matrix.append(str_i)
        #########Judgment of consistency#######
        val, vec = np.linalg.eig(judgment_matrix)
        val_list = list(abs(val))
        max_val = max(val_list)
        # CI
        CI = 1.0 * (max_val - nrow) / (nrow - 1)
        # RI:it is hard to give a proper weight matrix if n>=11
        RI = dict([(1, 0), (2, 0), (3, 0.58), (4, 0.90), (5, 1.12),
                   (6, 1.24), (7, 1.32), (8, 1.41), (9, 1.45), (10, 1.49)])
        # CR
        CR = 1.0 * CI / RI[nrow]
        if CR < 0.1:
            # normalize each columns
            M = np.zeros((nrow, ncol))
            for j in range(ncol):
                col_sum = np.sum(judgment_matrix[:, j])
                M[:, j] = 1.0 * judgment_matrix[:, j] / col_sum
            # mean each row
            C = []
            for i in range(nrow):
                C.append(np.mean(M[i, :]))
            C = C / sum(C)
            self.variable = {"method": "AHP",
                             "columns": columns_names,
                             "judgment_matrix": judgment_matrix,
                             "max_val": max_val,
                             "wide": ncol,
                             "CI": CI,
                             "RI": RI[ncol],
                             "CR": CR,
                             "weight": C}

import pandas as pd
a = AHP()

input_table_path = 'ahp-data1.xlsx'
# input_table_path:为excel的存储路径和名称
# 输入数据需为数值型数据
dataMat = pd.read_excel(input_table_path)
# 获取列名称，即变量名称
col_names = list(dataMat.columns)
#归一化
data = a.norm(dataMat)
allSum = np.zeros((data.shape[0],1))
for i in range(data.shape[0]):
    m = data[i]
    # 获取专家打分表
    scoreTable = a.getScoreTable(m)
#     print('专家打分表',scoreTable.shape)

    a.calculate_weight(col_names, scoreTable)
    b = a.variable
#     print (b['columns'], '的权重(w1,w2,....)分别为：', b['weight'])
    allSum[i] = np.dot(m,np.array(b['weight']));
    print(allSum[i])
print('发展指数平均值',np.mean(allSum))


[ 113.21812484]
发展指数平均值 113.218124835
